In [4]:
install.packages(paste("https://gecon.r-forge.r-project.org/files/win/3.6/gEcon_1.2.0.","zip",sep=""))
#install.packages(paste("http://grcon.r-forge.r-project.org/files/gEcon.iosam_0.2.0.","zip",sep=""))
#install.packages(paste("http://grcon.r-forge.r-project.org/files/gEcon.estimation_0.1.0.","zip",sep=""))

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Warning message:
“package ‘https://gecon.r-forge.r-project.org/files/win/3.6/gEcon_1.2.0.zip’ is not available (for R version 3.6.3)”


In [ ]:
library(gEcon)
library(gEcon.iosam)
library(gEcon.estiation)

In [ ]:
dsge_model <- gEcon::make_model("https://github.com/olgaklischuk/binder/dsge_model.gcn")

# solve the model####
dsge_model <- gEcon::steady_state(dsge_model)
dsge_model <- gEcon::solve_pert(dsge_model, loglin = TRUE)

In [ ]:
# set the stochastic shocks distribution parameters####
dsge_model <- gEcon::set_shock_distr_par(dsge_model,
                                  distr_par = list("sd( epsilon_G )" = 0.01,
                                                   "sd( epsilon_Z )" = 0.01))
gEcon::shock_info(model = dsge_model, all = TRUE)


In [ ]:
# ###################################################################
# 2. simulate the model to obtain data for the estimation###########

# choose variables of interest####
set.seed(250)
series_length <- 150
observables <- c("Y", "G")


In [ ]:
# simulate random path####
dsge_simulation <- gEcon::random_path(model = dsge_model,
                               sim_length = series_length,
                               variables = observables)
model_data <- gEcon::get_simulation_results(dsge_simulation)


In [ ]:
# create data set to be used for estimation (ts object)####
estimation_data <- ts(data = t(model_data)[, observables],
                      start = c(1973, 1),
                      frequency = 4, names = observables);estimation_data


In [ ]:
# remove mean from the data series####
mean_var <- matrix(apply(estimation_data, 2, mean),
                   byrow = TRUE,
                   nrow = nrow(estimation_data),
                   ncol = ncol(estimation_data));tail(mean_var)
estimation_data <- estimation_data  - mean_var



In [ ]:
# ###################################################################
# 3. declare prior distribution####

dsge_prior <- gEcon.estimation::gecon_prior(
  prior_list = list(
    list(par = "sd(epsilon_Z)", type = "inv_gamma",
         mean = 0.012, sd = 0.3, lower_bound = 0.0001,
         upper_bound  = 0.9, initial = 0.0012),
    list(par = "sd(epsilon_G)", type = "inv_gamma",
         mean = 0.008, sd = 0.3, lower_bound = 0.0001,
         upper_bound  = 0.9, initial = 0.006),
    list(par = "omega", type = "normal",
         mean = 1.45, sd = 0.1, lower_bound = 1,
         upper_bound  = 2, initial = 1.5),
    list(par = "phi_G", type = "beta",
         mean = 0.88, sd = 0.03, lower_bound = 0.5,
         upper_bound  = 0.999, initial = 0.95),
    list(par = "phi_Z", type = "beta",
         mean = 0.92, sd = 0.03, lower_bound = 0.5,
         upper_bound  = 0.999, initial = 0.95)),
  model = dsge_model)

gEcon.estimation::plot_prior((dsge_prior))+ggthemes::theme_economist_white()

In [ ]:
# ###################################################################
# 4. estimate the model (Bayesian estimation)####

estimation_result <- gEcon.estimation::bayesian_estimation(data_set = estimation_data[,1:2],
                                         optim_options_list = list(solver = "csminwel"),
                                         mcmc_options_list = list(chain_length = 1000,
                                                                  burn = 200,
                                                                  cores = 2, chains = 2,
                                                                  scale = rep(0.5, 5)),
                                         observables =  observables[1:2],
                                         model = dsge_model,
                                         prior = dsge_prior)

In [ ]:
gEcon.estimation::plot_posterior(estimation_result)

In [ ]:
# retrieve estimates####
#
# true model parameters were:
# sd(epsilon_Z) 0.01
# sd(epsilon_G) 0.01
# omega         1.45
# phi_G         0.9
# phi_Z         0.9
est_par <- gEcon.estimation::get_estimated_par(estimation_result)
free_par <- est_par$free_par;free_par
shock_distr_par <- est_par$shock_distr_par;shock_distr_par
estimated_dsge_model <- gEcon::set_free_par(dsge_model, free_par = free_par);estimated_dsge_model
estimated_dsge_model <- gEcon::set_shock_distr_par(estimated_dsge_model, distr_par = shock_distr_par)

estimated_dsge_model <- gEcon::steady_state(estimated_dsge_model)
estimated_dsge_model <- gEcon::solve_pert(estimated_dsge_model, loglin = TRUE)

In [ ]:
 ###################################################################
# 5. historical shock decomposition and variable smoothing####

# find historical shock decomposition
dsge_shock_decomp <- gEcon.estimation::shock_decomposition(model = estimated_dsge_model,
                                         data_set = window(estimation_data,
                                                           start = c(2004, 1),
                                                           end = c(2010, 1),
                                                           frequency = 4),
                                         observables = observables,
                                         variables = observables)
gEcon.estimation::plot_shock_decomposition(dsge_shock_decomp)


In [ ]:
# use Kalman smoother to obtain smoothed variables' values
dsge_smoothed_variables <- gEcon.estimation::smoother(model = estimated_dsge_model,
                                    data_set = estimation_data,
                                    observables = c("Y", "G"),
                                    variables = c("K", "I", "C"));dsge_smoothed_variables

# print smoothed shocks' values
dsge_smoothed_variables$smoothed_shock
# print smoothed variables' values
dsge_smoothed_variables$smoothed_var
# print the MSE matrix
dsge_smoothed_variables$MSE


In [ ]:

# ###################################################################
# 6. forecast using the model#######################################

# forecast using point estimates of parameters
fc_res <- gEcon.estimation::forecast(model = estimated_dsge_model,
                   data_set = estimation_data,
                   observables = observables,
                   variables = c("Y", "G"),
                   horizon = 20)

In [ ]:
# forecast using posterior distribution
fc_res_post <- gEcon.estimation::forecast_posterior(est_results = estimation_result,
                                  data_set = estimation_data,
                                  observables = observables,
                                  variables = c("Y", "G"),
                                  horizon = 20)

# plot forecasts
gEcon.estimation::plot_forecast(fc_res_post)
vplot_forecast(fc_res)

In [ ]:
d<-Sys.Date()
save.image(paste("https://github.com/olgaklischuk/binder/","dsge.",d,".RData",sep=""))
rm(list=ls())

In [3]:
d

[1] "2021-09-23"

In [5]:
print("Good bye!")

[1] "Good bye!"
